# Настройка pandas

In [1]:
# импортируем numpy и pandas
import numpy as np
import pandas as pd

# импортируем datetime
import datetime
from datetime import datetime, date

# задаем некоторые настройки pandas, регулирующие
# формат вывода
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 60)

# считываем данные в DataFrame, используя в качестве
# индекса столбец Symbol и записывая только те 
# столбцы, которые имеют позиции 0, 2, 3, 7
sp500 = pd.read_csv("C:/data/sp500.csv", 
                    index_col='Symbol', 
                    usecols=[0, 2, 3, 7])

# Важность применения индексов

In [2]:
# создаем DataFame, состоящий из случайных чисел и столбца key
np.random.seed(123456)
df = pd.DataFrame({'foo':np.random.random(10000), 'key':range(100, 10100)})
df[:5]

        foo  key
0  0.126970  100
1  0.966718  101
2  0.260476  102
3  0.897237  103
4  0.376750  104

In [3]:
# отбираем строку, в котором значение столбца key равно 10099
df[df.key==10099]

           foo    key
9999  0.272283  10099

In [4]:
# измеряем время выполнения операции отбора
%timeit df[df.key==10099]

1000 loops, best of 3: 572 µs per loop


In [5]:
# превращаем столбец key в index
df_with_index = df.set_index(['key'])
df_with_index[:5]

          foo
key          
100  0.126970
101  0.966718
102  0.260476
103  0.897237
104  0.376750

In [6]:
# теперь можно найти это значение
# с помощью индекса
df_with_index.loc[10099]

foo    0.272283
Name: 10099, dtype: float64

In [7]:
# и теперь операция выполняется намного быстрее
%timeit df_with_index.loc[10099]

10000 loops, best of 3: 115 µs per loop


# Основной тип Index

In [8]:
# покажем, что столбцы фактически являются индексом
temps = pd.DataFrame({ "City": ["Missoula", "Philadelphia"],
                       "Temperature": [70, 80] })
temps

           City  Temperature
0      Missoula           70
1  Philadelphia           80

In [9]:
# мы видим, что столбцы - это индекс
temps.columns

Index(['City', 'Temperature'], dtype='object')

# Индексы Int64Index и RangeIndex, в качестве меток используются целые числа

In [10]:
# явно создаем Int64Index
df_i64 = pd.DataFrame(np.arange(10, 20), index=np.arange(0, 10))
df_i64[:5]

    0
0  10
1  11
2  12
3  13
4  14

In [11]:
# смотрим индекс
df_i64.index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')

In [12]:
# по умолчанию мы получаем RangeIndex
df_range = pd.DataFrame(np.arange(10, 15))
df_range[:5]

    0
0  10
1  11
2  12
3  13
4  14

In [13]:
df_range.index

RangeIndex(start=0, stop=5, step=1)

# Индекс Float64Index, в качестве меток используются числа с плавающей точкой

In [14]:
# индексы, использующие Float64Index
df_f64 = pd.DataFrame(np.arange(0, 1000, 5), 
                      np.arange(0.0, 100.0, 0.5))
df_f64.iloc[:5] # потребуется iloc для отбора первых пяти строк

      0
0.0   0
0.5   5
1.0  10
1.5  15
2.0  20

In [15]:
df_f64.index

Float64Index([ 0.0,  0.5,  1.0,  1.5,  2.0,  2.5,  3.0,
               3.5,  4.0,  4.5,
              ...
              95.0, 95.5, 96.0, 96.5, 97.0, 97.5, 98.0,
              98.5, 99.0, 99.5],
             dtype='float64', length=200)

# Представление дискретных интервалов с использованием IntervalIndex

In [16]:
# датафрейм с IntervalIndex
df_interval = pd.DataFrame({ "A": [1, 2, 3, 4]},
                    index = pd.IntervalIndex.from_breaks(
                        [0, 0.5, 1.0, 1.5, 2.0]))
df_interval

            A
(0.0, 0.5]  1
(0.5, 1.0]  2
(1.0, 1.5]  3
(1.5, 2.0]  4

In [17]:
df_interval.index

IntervalIndex([(0.0, 0.5], (0.5, 1.0], (1.0, 1.5], (1.5, 2.0]]
              closed='right',
              dtype='interval[float64]')

# Категории в качестве индекса – CategoricalIndex

In [18]:
# создаем датафрейм со столбцом, имеющим тип Categorical
df_categorical = pd.DataFrame({'A': np.arange(6),
                               'B': list('aabbca')})
df_categorical['B'] = df_categorical['B'].astype('category', 
                                          categories=list('cab'))
df_categorical

   A  B
0  0  a
1  1  a
2  2  b
3  3  b
4  4  c
5  5  a

In [19]:
# превращаем категориальный столбец в индекс
df_categorical = df_categorical.set_index('B')
df_categorical.index

CategoricalIndex(['a', 'a', 'b', 'b', 'c', 'a'], categories=['c', 'a', 'b'], ordered=False, name='B', dtype='category')

In [20]:
# ищем значения в категории 'a'
df_categorical.loc['a']

   A
B   
a  0
a  1
a  5

# Индексирование по датам и времени с помощью DatetimeIndex

In [21]:
# создаем DatetimeIndex на основе диапазона дат
rng = pd.date_range('5/1/2017', periods=5, freq='H')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2017-05-01 00:00:00    1.239792
2017-05-01 01:00:00   -0.400611
2017-05-01 02:00:00    0.718247
2017-05-01 03:00:00    0.430499
2017-05-01 04:00:00    1.155432
Freq: H, dtype: float64

In [22]:
ts.index

DatetimeIndex(['2017-05-01 00:00:00',
               '2017-05-01 01:00:00',
               '2017-05-01 02:00:00',
               '2017-05-01 03:00:00',
               '2017-05-01 04:00:00'],
              dtype='datetime64[ns]', freq='H')

# Индексирование периодов времени с помощью PeriodIndex

In [23]:
# явно создаем PeriodIndex
periods = pd.PeriodIndex(['2017-1', '2017-2', '2017-3'], freq='M')
periods

PeriodIndex(['2017-01', '2017-02', '2017-03'], dtype='period[M]', freq='M')

In [24]:
# используем индекс в серии
period_series = pd.Series(np.random.randn(len(periods)), 
                          index=periods)
period_series

2017-01   -0.449276
2017-02    2.472977
2017-03   -0.716023
Freq: M, dtype: float64

# Создание и использование индекса в объекте Series или объекте DataFrame

In [25]:
# создаем DatetimeIndex
date_times = pd.DatetimeIndex(pd.date_range('5/1/2017', 
                                            periods=5, 
                                            freq='H'))
date_times

DatetimeIndex(['2017-05-01 00:00:00',
               '2017-05-01 01:00:00',
               '2017-05-01 02:00:00',
               '2017-05-01 03:00:00',
               '2017-05-01 04:00:00'],
              dtype='datetime64[ns]', freq='H')

In [26]:
# создаем объект DataFrame, используя индекс
df_date_times = pd.DataFrame(np.arange(0, len(date_times)), 
                             index=date_times)
df_date_times

                     0
2017-05-01 00:00:00  0
2017-05-01 01:00:00  1
2017-05-01 02:00:00  2
2017-05-01 03:00:00  3
2017-05-01 04:00:00  4

In [27]:
# задаем индекс датафрейма
df_date_times.index = pd.DatetimeIndex(pd.date_range('6/1/2017', 
                                                     periods=5, 
                                                     freq='H'))
df_date_times

                     0
2017-06-01 00:00:00  0
2017-06-01 01:00:00  1
2017-06-01 02:00:00  2
2017-06-01 03:00:00  3
2017-06-01 04:00:00  4

# Отбор значений с помощью индекса

In [28]:
# создаем серию
s = pd.Series(np.arange(0, 5), index=list('abcde'))
s

a    0
b    1
c    2
d    3
e    4
dtype: int32

In [29]:
# ищем по метке индекса
s['b']

1

In [30]:
# явно ищем по метке индекса
s.loc['b']

1

In [31]:
# создаем датафрейм с двумя столбцами
df = pd.DataFrame([ np.arange(10, 12), 
                    np.arange(12, 14)], 
                  columns=list('ab'), 
                  index=list('vw'))
df

    a   b
v  10  11
w  12  13

In [32]:
# эта строка возвращает столбец 'a'
df['a']

v    10
w    12
Name: a, dtype: int64

In [33]:
# возвращает строку 'w' по метке
df.loc['w']

a    12
b    13
Name: w, dtype: int64

In [34]:
# создаем срез серии от метки b до метки d
s['b':'d']

b    1
c    2
d    3
dtype: int32

In [35]:
# эта строка явно создает срез от метки b до метки d
s.loc['b':'d']

b    1
c    2
d    3
dtype: int32

In [36]:
# ищем строки по метке
s.loc[['a', 'c', 'e']]

a    0
c    2
e    4
dtype: int32

# Преобразование данных в индекс и получение данных из индекса 

In [37]:
# исследуем несколько строк датафрейма sp500
sp500[:5]

                        Sector   Price  Book Value
Symbol                                            
MMM                Industrials  141.14      26.668
ABT                Health Care   39.60      15.573
ABBV               Health Care   53.95       2.954
ACN     Information Technology   79.79       8.326
ACE                 Financials  102.91      86.897

In [38]:
# сбрасываем индекс, помещая значения 
# индекса в столбец
index_moved_to_col = sp500.reset_index()
index_moved_to_col[:5]

  Symbol                  Sector   Price  Book Value
0    MMM             Industrials  141.14      26.668
1    ABT             Health Care   39.60      15.573
2   ABBV             Health Care   53.95       2.954
3    ACN  Information Technology   79.79       8.326
4    ACE              Financials  102.91      86.897

In [39]:
# а теперь делаем столбец Sector индексом
index_moved_to_col.set_index('Sector')[:5]

                       Symbol   Price  Book Value
Sector                                           
Industrials               MMM  141.14      26.668
Health Care               ABT   39.60      15.573
Health Care              ABBV   53.95       2.954
Information Technology    ACN   79.79       8.326
Financials                ACE  102.91      86.897

In [40]:
# делаем переиндексацию, задав метки MMM, ABBV и FOO
reindexed = sp500.reindex(index=['MMM', 'ABBV', 'FOO'])
# обратите внимание, что ABT и ACN удалены, а FOO содержит значения NaN
reindexed

             Sector   Price  Book Value
Symbol                                 
MMM     Industrials  141.14      26.668
ABBV    Health Care   53.95       2.954
FOO             NaN     NaN         NaN

In [41]:
# выполняем переиндексацию столбцов
sp500.reindex(columns=['Price', 
                       'Book Value', 
                       'NewCol'])[:5]

         Price  Book Value  NewCol
Symbol                            
MMM     141.14      26.668     NaN
ABT      39.60      15.573     NaN
ABBV     53.95       2.954     NaN
ACN      79.79       8.326     NaN
ACE     102.91      86.897     NaN

# Иерархическая индексация

In [42]:
# сначала помещаем символы в столбец
reindexed = sp500.reset_index()
# а теперь индексируем датафрейм sp500 по столбцам Sector и Symbol
multi_fi = reindexed.set_index(['Sector', 'Symbol'])
multi_fi[:5]

                                Price  Book Value
Sector                 Symbol                    
Industrials            MMM     141.14      26.668
Health Care            ABT      39.60      15.573
                       ABBV     53.95       2.954
Information Technology ACN      79.79       8.326
Financials             ACE     102.91      86.897

In [43]:
# наш индекс - это MultiIndex
type(multi_fi.index)

pandas.core.indexes.multi.MultiIndex

In [44]:
# он имеет два уровня
len(multi_fi.index.levels)

2

In [45]:
# каждый уровень индекса - это индекс
multi_fi.index.levels[0]

Index(['Consumer Discretionary', 'Consumer Discretionary ',
       'Consumer Staples', 'Consumer Staples ', 'Energy',
       'Financials', 'Health Care', 'Industrials',
       'Industries', 'Information Technology', 'Materials',
       'Telecommunications Services', 'Utilities'],
      dtype='object', name='Sector')

In [46]:
# каждый уровень индекса - это индекс
multi_fi.index.levels[1]

Index(['A', 'AA', 'AAPL', 'ABBV', 'ABC', 'ABT', 'ACE',
       'ACN', 'ACT', 'ADBE',
       ...
       'XLNX', 'XOM', 'XRAY', 'XRX', 'XYL', 'YHOO', 'YUM',
       'ZION', 'ZMH', 'ZTS'],
      dtype='object', name='Symbol', length=500)

In [47]:
# значения в уровне индекса 0
multi_fi.index.get_level_values(0)

Index(['Industrials', 'Health Care', 'Health Care',
       'Information Technology', 'Financials',
       'Health Care', 'Information Technology',
       'Utilities', 'Health Care', 'Financials',
       ...
       'Utilities', 'Information Technology',
       'Information Technology', 'Financials',
       'Industrials', 'Information Technology',
       'Consumer Discretionary', 'Health Care',
       'Financials', 'Health Care'],
      dtype='object', name='Sector', length=500)

In [48]:
# получаем все акции, которые имеют значение Industrials
# обратите внимание, что в результатах 
# индекс уровня 0 не выводится 
multi_fi.xs('Industrials')[:5]

         Price  Book Value
Symbol                    
MMM     141.14      26.668
ALLE     52.46       0.000
APH      95.71      18.315
AVY      48.20      15.616
BA      132.41      19.870

In [49]:
# отбираем строки, в которых индекс уровня 1 
# (Symbol) имеет значение ALLE
# обратите внимание, что в результатах 
# индекс уровня 1 (Symbol) не выводится 
multi_fi.xs('ALLE', level=1)

             Price  Book Value
Sector                        
Industrials  52.46         0.0

In [50]:
# отбираем строки, в которых индекс уровня 0 
# (Sector) имеет значение Industrials, 
# без удаления уровней
multi_fi.xs('Industrials', drop_level=False)[:5]

                     Price  Book Value
Sector      Symbol                    
Industrials MMM     141.14      26.668
            ALLE     52.46       0.000
            APH      95.71      18.315
            AVY      48.20      15.616
            BA      132.41      19.870

In [51]:
# скомбинируем уровни индексов
multi_fi.xs('Industrials').xs('UPS')

Price         102.73
Book Value      6.79
Name: UPS, dtype: float64

In [52]:
# комбинируем уровни индексов, используя кортеж
multi_fi.xs(('Industrials', 'UPS'))

Price         102.73
Book Value      6.79
Name: (Industrials, UPS), dtype: float64